In [411]:
from functools import reduce
import statistics as st
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
#import matplotlib.pyplot as plt
import os
#from PyPDF2 import PdfReader
#%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\epenago\AppData\Local\Tesseract-OCR\tesseract.exe'

In [412]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

In [413]:
def calcular_espacios(diccionario, indices):
    
    suma = 0
    bandera = True
  
    width_espacios = []
    for i in range(len(indices)):
             
        if(bandera == True):
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            if i < len(indices)-1:
                if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                    pass
                else:
                    bandera = True
            else:           
                    bandera = True
                    

        elif(i < len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                pass
            else:
                bandera = True
               

        elif(i <= len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            bandera = True
            
    #print(width_espacios)
    width_espacios.sort() 
    
    moda = st.mode(width_espacios)
    return (moda-(moda*1), moda+(moda*1))

In [414]:
def coordenadas_extraccion(diccionario, indices, intervalo):
    
    datos_extraidos = []
    dict = {}
    coordenadas = []
    suma = 0
    bandera = True
    indice_anterior = -1
    strings = []
    string = ""
    index = []
    line = 0

    for i in range(len(indices)):
             
        if(bandera == True):
            indice_anterior = i
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            # Se guarda la palabra y su indice
            string = diccionario["text"][indices[i]]
            index.append(indices[i])
            
            if i < len(indices)-1:
                if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                    
                    if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                        bandera = True
                        #print(indices[i])
                        coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                        diccionario["top"][indices[indice_anterior]], 
                        suma,
                        diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                        
                        dict["text"] = string
                        dict["line"] = line
                        dict["index"] = index
                        dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                        datos_extraidos.append(dict)
                                                                    
                        dict = {}                        
                        string = ""
                        index = []

                        bandera = True
                        indice_anterior = -1
                else:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                    

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []
                    line += 1

                    bandera = True
                    indice_anterior = -1
            else:           
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1

        elif(i < len(indices)-1):
            string += (" " + diccionario["text"][indices[i]])
            index.append(indices[i])

            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                
                if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1
            else:
                bandera = True
                #print(indices[i])
                coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                diccionario["top"][indices[indice_anterior]], 
                suma,
                diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                dict["text"] = string
                dict["line"] = line
                dict["index"] = index
                dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                datos_extraidos.append(dict)
                                                            
                dict = {}                        
                string = ""
                index = []
                line += 1

                bandera = True
                indice_anterior = -1

        elif(i <= len(indices)-1):
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            coordenadas.append([diccionario["left"][indices[indice_anterior]], 
            diccionario["top"][indices[indice_anterior]], 
            suma,
            diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

            string = ""
            index = []

            bandera = True
            indice_anterior = -1

    return (coordenadas, datos_extraidos)

En el Metodo extraccion_columns logramos separar las columnas extrayedo y clasificando los indices en el dicionario data_ordenaizer

In [415]:
def extracion_colums(df):

    arr = np.array(df["coordinates"].to_list())
    colums = []
    
    for i in np.arange(len(arr)):
        temp = []
        temp.append(i)
        for j in np.arange(len(arr)):

            if i == j: continue
            interval1 = np.arange(arr[i,0], arr[i,1]+1, dtype=int)
            interval2 = np.arange(arr[j,0], arr[j,1]+1, dtype=int)

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 55 and percent2 > 55:
                temp.append(j)
        
        colums.append(temp)
        
    cl = []
    descartados = []
    for i in np.arange(len(colums)):
        temp = []
        temp = colums[i]
        if i in descartados: continue
        for j in np.arange(len(colums)):

            if i == j: continue
            interval1 = colums[i]
            interval2 = colums[j]

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 20 or percent2 > 20:
                temp += colums[j]
                descartados.append(j)
        cl.append(temp)

    colums = []    
    #se eliminan elementos repetidos en las columnas
    for element in cl:
        temp = []
        for i in element:
            if i not in temp:
                temp.append(i)
        colums.append(temp)
    

    print("el tamaño de cl es: ", len(cl))
    #en este bloque se saca la pocision en pixeles de la imagen
    min_max = []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max.append((menor,mayor))

    for i in range(len(min_max)):
        print(min_max[i], colums[i])

    cl = []
    descartados = []
    by_delete = []
    for i in range(len(min_max)):
        temp= []
        temp= colums[i]
        if i in descartados: continue
        for j in range(len(min_max)):

            if i==j: continue
            if min_max[i][0]-10 <= min_max[j][0] and min_max[i][1]+10 >= min_max[j][1]:
                temp += colums[j]
                descartados.append(j)
                if i > j:
                    by_delete.append(colums[j])
        cl.append(temp)

    colums = cl
    print("estos son a eliminar", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    print()
    zises = []
    for element in colums:
        print(element)
        zises.append(len(element))
    
    mean = st.mean(zises)
    print("la media de los len ", mean)
    by_delete = []
    for i in range(len(colums)):
        percent20 = (mean/100)*20
        #print("percent20: ", percent20)
        if len(colums[i]) < percent20:
            by_delete.append(colums[i])

    for element in colums:
        print(element)
        print

    print("elementos a eliminar: ", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    for element in colums:
        print(element)

    min_max_h= []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max_h.append((menor,mayor))

    min_max_h_unOrder = min_max_h.copy()
    min_max_h.sort()

    cl = []
    for i in np.arange(len(colums)):
        for j in np.arange(len(colums)):
            if min_max_h_unOrder[j] == min_max_h[i]:
                cl.append(colums[j])

    colums = cl

    print(min_max_h)
    
    return min_max_h,colums

El metodo build_df toma el arreglo con las columnas separadar y utiliza el diccionario construido en extracion_colums 
para crear un objeto tipo pandas.DataFrame con la misma distribucion de los datos que estaba en la imagen

In [416]:
def build_df(pd_dataOrdenizer, colums):
    temp = pd.DataFrame()
    pd_columns = pd.DataFrame()
    for i in range(len(colums)):
        temp = pd.DataFrame.copy(pd_dataOrdenizer.loc[colums[i], ["text", "line"]])
        temp = temp.drop_duplicates()
        temp = temp.sort_values("line")
        temp = pd.Series(temp["text"].tolist(), index=temp["line"].tolist())
        temp = temp.loc[~temp.index.duplicated(keep='first')]
        pd_columns = pd.concat([pd_columns, temp], axis=1, ignore_index=True)

    pd_columns = pd_columns.sort_index()
    return pd_columns

In [417]:
def del_headEnd(df):

    freq = df['line'].value_counts().sort_index().head(10)
    to_delete = []
    bandera = True

    for i in range(len(freq)):
        if bandera == True:
            if freq[i] == 1:
                to_delete.append(i)
            
            elif freq[i] == 2 and freq[i+1] == 1:
                to_delete.append(i)
            else: bandera = False
    
    for i in to_delete:
        df.drop(df.index[df['line'] == i] , axis=0, inplace=True)
    df.reset_index(inplace=True)
    df.drop(['level_0'], axis=1, inplace=True)
    

In [418]:
def isTime(string):
    try:
        date = datetime.strptime(string	, '%d/%m/%Y')
        return str(date.year)
    except ValueError:
        return np.nan


In [419]:
def isNumeric(s):
    try:
        if type(s) != float: #se pregunta si el dato no es NaN
            s = s.replace('(','') 
            s = s.replace(')','') 
            s = s.replace(',','')
            s = s.replace('.','')
        float(s)
        return True
    except ValueError:
        return False

In [420]:
def convert_parenthesis(s):
   
    if type(s) != float:
        if '(' in s or ')' in s:
            s = s.replace('(','') 
            s = s.replace(')','')
            s = '-' + s
    
    return s


In [421]:
def isNumericColum(df): 
    type_index=[]                   
    str_index=[]
    counter_num = 1

    for i in df.columns:
        df[i] = df.apply(lambda x: x[i].replace(',','') if type(x[i]) == str else x[i], axis=1)
        df[i] = df.apply(lambda x: x[i].replace('.','') if type(x[i]) == str else x[i], axis=1)
        df[i] = df.apply(lambda x: '0' if x[i] == 'o' else x[i], axis=1)

    for i in df.columns:
        size = df[i].apply(isNumeric).value_counts().sum()
        try:
            if 100*df[i].apply(isNumeric).value_counts()[True]/size >= 80:
                type_index.append('num')
                df[i] = df[i].apply(convert_parenthesis)
            else:
                type_index.append('text')
            
        except KeyError:
            type_index.append('text')

    years = np.arange(2018,datetime.now().year + 1)
    years = [str(i) for i in years]
    years.append('NOTAS')
    for i in df.columns:
        if type_index[i] == 'text':
            str_index.append(df[i][~df[i].isnull()].head(1).to_list()[0])
            #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])
        else:
            
            df_three = df[i][~df[i].isnull()].head(3).apply(lambda x: x if x.upper() in years else np.nan)
            if df_three.count() == 0:
                df_three = df[i][~df[i].isnull()].head(3).apply(isTime)
                if df_three.count() == 0:
                    if len(df[i][~df[i].isnull()].head(3)[~df[i][~df[i].isnull()].apply(isNumeric).head(3)].to_list()) == 0:
                        str_index.append('num' + str(counter_num))
                        counter_num += 1
                        
                    else:
                        str_index.append(df[i][~df[i].isnull()].head(3)[~df[i][~df[i].isnull()].apply(isNumeric).head(3)].to_list()[0])

                else:
                    str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
            else:
                str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
                #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])

    str_index = [i.upper() for i in str_index]

    return str_index, type_index

In [422]:
def isNan(x):
    try:
        return np.nan(x)
    except TypeError:
        return False

In [423]:
def conver_to_float(x):
    
    try:
        if isNumeric(x) and isNan(x) == False:
            x = convert_parenthesis(x)
            if ',' in x and '.' in x:

                    x = x.replace(',','.')
                    num_decimales = len(x.split('.')[-1]) 
                    x = x.replace('.','',x.count('.')-1)
            
                    
                    #x = float(x)
            elif len(x.replace(',','.').split('.')[-1]) != 3 or len(x.replace('.',',').split('.')[-1]) != 3: 
                    x = x.replace(',','.')
                    num_decimales = len(x.split('.')[-1])
                    if num_decimales != 3:
                         x = x.replace('.','',x.count('.')-1)
                    else:
                        x = x.replace('.','')

            else:
                x = x.replace(',','')
                x = x.replace('.','')
                #x = float(x)
                
        return x
    except TypeError:
        return x

    

In [461]:
#main
#path = r'./reto/1.PNG'
path = r'.\reto\22.PNG'
image = cv2.imread(path)
gray = get_grayscale(image)
th = thresholding(gray)
openingg = opening(gray)
cannyy = canny(gray)
img = image
hImg, wImg, _ = img.shape

custom_config = r'-c tessedit_char_blacklist=$¢§_{} --oem 3 --psm 4' 
data = pytesseract.image_to_data(img, lang= 'eng+spa', config=custom_config, output_type= 'dict')


In [462]:
pd.DataFrame(data).tail(50)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
198,5,1,5,1,7,4,550,1653,77,25,96.625183,renta
199,5,1,5,1,7,5,1191,1656,193,33,96.165825,"(12,473,000)"
200,5,1,5,1,7,6,1684,1659,192,34,96.624390,"(32,997,496)"
201,4,1,5,1,8,0,201,1692,417,33,-1,
202,5,1,5,1,8,1,201,1692,139,33,82.839920,impuesto
203,5,1,5,1,8,2,343,1688,16,43,96.937401,a
204,5,1,5,1,8,3,381,1693,25,27,96.974289,la
205,5,1,5,1,8,4,418,1696,75,25,96.992592,renta
206,5,1,5,1,8,5,506,1694,112,27,96.963890,diferido
207,2,1,6,0,0,0,197,1735,1671,42,-1,


In [463]:
indice_words = []
for i in range(len(data['text'])):
    if data['text'][i] != '' and data['text'][i] != ' ' and float(data['conf'][i]) >= 50 and data["text"][i] != "-" and data["text"][i] != "--" and data["text"][i] != "s" and data["text"] != "S":
        indice_words.append(i)

#print(len(indice_words))
#print(indice_words)
intervalo = calcular_espacios(data, indice_words)
print(intervalo)
coordenadas,data_organizer = coordenadas_extraccion(data, indice_words, intervalo)
df_data_organizer = pd.DataFrame(data_organizer)

(0, 22)


In [464]:
df_data_organizer.head(50)

,text,line,index,coordinates
0,ANO,0,[4],"(1269, 1317)"
1,2018,0,[5],"(1353, 1394)"
2,ANO,0,[6],"(1762, 1811)"
3,2019,0,[7],"(1847, 1882)"
4,INGRESOS,0,[9],"(219, 397)"
5,"1,227,511,697",1,[10],"(1175, 1392)"
6,"2,033,343,018",1,[11],"(1667, 1886)"
7,ingresos por ventas,2,"[13, 14, 15]","(210, 510)"
8,"1,237,711,697",2,[16],"(1176, 1391)"
9,"2,033,343,018",2,[17],"(1666, 1886)"


In [465]:
df_data_organizer.loc[:,"coordinates"].apply(lambda x: (x[0]+x[1])/2).head(10)

0    1293.0
1    1373.5
2    1786.5
3    1864.5
4     308.0
5    1283.5
6    1776.5
7     360.0
8    1283.5
9    1776.0
Name: coordinates, dtype: float64

In [466]:
del_headEnd(df_data_organizer)

In [467]:
df_data_organizer.head(5)

,text,line,index,coordinates
0,ANO,0,[4],"(1269, 1317)"
1,2018,0,[5],"(1353, 1394)"
2,ANO,0,[6],"(1762, 1811)"
3,2019,0,[7],"(1847, 1882)"
4,INGRESOS,0,[9],"(219, 397)"


In [468]:

#print("numero de posiciones: " + str(len(coordenadas)))
#del_headEnd(df_data_organizer)
min_max_h,colums = extracion_colums(df_data_organizer)

#Dibujado de los rectangulos en la imagen
for p in min_max_h:
    cv2.rectangle(img, (p[0], hImg-5),(p[1], 5), (random.randint(0,255), random.randint(0,255), random.randint(0,255)), 1)

for p in coordenadas:
    cv2.rectangle(img, (p[0], p[1]),(p[2], p[3]), (50, 50, 255), 1)

cv2.imwrite('salida.png',img)

df = pd.DataFrame(build_df(df_data_organizer , colums))

df_copy = df.copy()


index, type_index = isNumericColum(df_copy)
for i in df.columns:
    if type_index[i] == 'num':
        df[i] = df[i].apply(conver_to_float)

for i in df.columns:
    if type_index[i] == 'num':
        df[i] = pd.to_numeric(df[i], errors='coerce')
df.columns = index

df.to_excel("data.xlsx", index=False)


"""
img = cv2.resize(img, (600, 700))
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

el tamaño de cl es:  15
(1269, 1317) [0]
(1331, 1394) [1, 92]
(1762, 1811) [2]
(1846, 1885) [3, 12]
(197, 876) [4, 7, 16, 20, 23, 34, 37, 40, 46, 49, 52, 58, 60, 62, 64, 75, 77, 82, 91, 104, 10, 13, 26, 28, 31, 43, 54, 56, 67, 73, 79, 97, 100, 14, 70, 85, 88, 94, 101, 106, 111, 108]
(1175, 1392) [5, 8, 11, 18, 24, 27, 29, 32, 41, 50, 55, 65, 68, 80, 83, 86, 95, 98, 102, 38, 89]
(1643, 1886) [6, 9, 15, 19, 25, 30, 33, 39, 42, 51, 66, 69, 81, 84, 93, 96, 99, 103, 90]
(980, 982) [17]
(1368, 1389) [21, 35, 44, 47, 53, 57, 59, 61, 63, 72, 74, 76, 78]
(1861, 1883) [22, 36, 45, 48, 87]
(1043, 1047) [71]
(482, 499) [105]
(1494, 1781) [107, 110, 112]
(616, 704) [109]
(1726, 1747) [113]
estos son a eliminar [[0], [1, 92, 21, 35, 44, 47, 53, 57, 59, 61, 63, 72, 74, 76, 78], [2], [3, 12, 22, 36, 45, 48, 87]]

[4, 7, 16, 20, 23, 34, 37, 40, 46, 49, 52, 58, 60, 62, 64, 75, 77, 82, 91, 104, 10, 13, 26, 28, 31, 43, 54, 56, 67, 73, 79, 97, 100, 14, 70, 85, 88, 94, 101, 106, 111, 108, 105, 109]
[5, 8, 1

"\nimg = cv2.resize(img, (600, 700))\ncv2.imshow('Result', img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [469]:
type_index

['text', 'num', 'num']

In [470]:
index

['INGRESOS', '2018', 'ANO']

In [471]:
df

,INGRESOS,2018,ANO
0,INGRESOS,2.018000e+03,NaN
1,NaN,1.227512e+09,2.033343e+09
2,ingresos por ventas,1.237712e+09,2.033343e+09
3,Devoluciones y descuentos en,-1.020000e+07,-0.000000e+00
4,COSTO DE VENTAS,NaN,NaN
5,COSTO DE MERCANCIA VENDIDA,NaN,-1.595728e+09
6,Costo de ventas,NaN,NaN
8,NaN,9.896421e+08,1.595728e+09
9,Diversos,0.000000e+00,0.000000e+00
10,UTILIDAD BRUTA,2.378695e+08,4.376148e+08


In [472]:
df.dtypes

INGRESOS     object
2018        float64
ANO         float64
dtype: object